In [13]:
# import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature


In [14]:
## Load the dataset
X,y = datasets.load_iris(return_X_y=True)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

## define the model hyperparameters
params = {"penalty": "l2", "solver": "lbfgs", "max_iter": 1000, "multi_class": "auto", "random_state": 1000}

## train the model
model = LogisticRegression(**params)
model.fit(X_train,y_train)


D:\dev\git-hub\python\ml-ops-learning\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=1000)

In [15]:
## predict the test set
y_pred = model.predict(X_test)

In [16]:
## check the accuracy
accuracy = accuracy_score(y_test,y_pred)
print(accuracy)

1.0


In [17]:
## mlflow tracking
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.models.signature import infer_signature

# Set both tracking and registry URI to the same port (5001 since 5000 was in use)
mlflow.set_tracking_uri("http://localhost:5000")

try:
    # Create a unique experiment
    experiment_name = "iris_model_tracking"
    mlflow.set_experiment(experiment_name)
    print(f"Successfully created experiment: {experiment_name}")

    # Start a new run and log everything within this context
    with mlflow.start_run(description="Training iris model") as run:
        print(f"Started new run: {run.info.run_id}")

        # Log the parameters and metrics
        mlflow.log_params(params)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.set_tag("Training Info", "Basic LR model for iris data")

        # Log the model with signature
        signature = infer_signature(X_train, model.predict(X_train))
        model_info = mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path="iris_model",
            signature=signature,
            input_example=X_train,
            registered_model_name="iris-tracking-1"
        )

        print(f"Model logged successfully with info: {model_info}")

except Exception as e:
    print(f"Error: {str(e)}")

2025/02/18 23:01:50 INFO mlflow.tracking.fluent: Experiment with name 'iris_model_tracking' does not exist. Creating a new experiment.


Successfully created experiment: iris_model_tracking
Started new run: a39835da17244f8e825e79e70a1e1cdb


Successfully registered model 'iris-tracking-1'.
2025/02/18 23:01:56 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-tracking-1, version 1


Model logged successfully with info: <mlflow.models.model.ModelInfo object at 0x0000022790BE72D0>
🏃 View run hilarious-mink-723 at: http://localhost:5000/#/experiments/955876357238564658/runs/a39835da17244f8e825e79e70a1e1cdb
🧪 View experiment at: http://localhost:5000/#/experiments/955876357238564658


Created version '1' of model 'iris-tracking-1'.


In [19]:
## Validating the model

import mlflow
import pandas as pd

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)
prediction=loaded_model.predict(X_test)

iris_features_name=datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_features_name)
result["actual_class"]  = y_test
result["prediction"] = prediction
result


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,prediction
0,5.7,2.8,4.5,1.3,1,1
1,5.1,3.4,1.5,0.2,0,0
2,7.0,3.2,4.7,1.4,1,1
3,4.4,3.2,1.3,0.2,0,0
4,6.4,2.7,5.3,1.9,2,2
5,5.7,4.4,1.5,0.4,0,0
6,5.0,3.4,1.6,0.4,0,0
7,6.5,3.2,5.1,2.0,2,2
8,5.1,3.8,1.5,0.3,0,0
9,6.4,3.2,5.3,2.3,2,2
